In [ ]:
import re
from pymongo import MongoClient
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string

In [ ]:
# Membuat koneksi ke database MongoDB
client = MongoClient('localhost', 27017)
db = client['google_play_store_data'] 
raw_collection = db['01. raw_data_bibit']   
prepo_collection = db['02. prepo_data_bibit'] 

# Mengambil beberapa dokumen dari koleksi lama
data = raw_collection.find({})  # Misalnya, mengambil semua dokumen

# Menyimpan dokumen-dokumen tersebut ke dalam list
data_list = list(data)

# Menyimpan data ke dalam koleksi baru
prepo_collection.insert_many(data_list)


In [ ]:
# Menghapus field yang tidak diperlukan
prepo_collection.update_many({}, {'$unset': {'reviewId': '', 'userName': '', 'userImage': '', 'score': '',
                                       'thumbsUpCount': '', 'reviewCreatedVersion': '','replyContent': '',
                                       'repliedAt': '', 'appVersion': ''}})

# Menampilkan data setelah penghapusan field
cursor = prepo_collection.find()
for document in cursor:
    print(document)

In [ ]:
def count_changes(original_text, modified_text):
    count = sum(1 for i, j in zip(original_text, modified_text) if i != j)
    return count

def lowercase(text):
    modified_text = text.lower()
    changes_count = count_changes(text, modified_text)
    return modified_text, changes_count

cursor = prepo_collection.find()

total_lower_count = 0

for document in cursor:
    review = document['content']
    lower_review, lower_count = lowercase(review)

# Memperbarui dokumen dengan review yang sudah bersih
    prepo_collection.update_one({'_id': document['_id']}, {'$set': {'content': lower_review}})

    total_lower_count += lower_count

# Menampilkan jumlah karakter non-ascii yang dihapus
print(f"Total jumlah huruf yang diubah menjadi huruf kecil: {total_lower_count}")


In [ ]:
def remove_ascii(text):
    modified_text = text.encode('ascii', 'replace').decode('ascii')
    changes_count = count_changes(text, modified_text)
    return modified_text, changes_count

cursor = prepo_collection.find()

total_cleaned_count = 0

for document in cursor:
    review = document['content']
    cleaned_review, cleaned_count = remove_ascii(review)

    # Memperbarui dokumen dengan review yang sudah bersih
    prepo_collection.update_one({'_id': document['_id']}, {'$set': {'content': cleaned_review}})

    total_cleaned_count += cleaned_count

# Menampilkan jumlah karakter non-ascii yang dihapus
print(f"Jumlah karakter non-ASCII yang dihapus: {total_cleaned_count}")


In [ ]:
def remove_punctuation(text):
    # Pola regex untuk tanda baca
    punctuation_pattern = r'[' + re.escape(string.punctuation) + ']'
    # [!"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}\~]

    # Menghapus tanda baca menggunakan regex
    modified_text = re.sub(punctuation_pattern, '', text)
    changes_count = count_changes(text, modified_text)
    return modified_text, changes_count

# Mengambil semua dokumen dari koleksi
documents = prepo_collection.find()

total_cleaned_count = 0

for document in documents:
    review = document['content']
    cleaned_review, cleaned_count = remove_punctuation(review)
    
    # Memperbarui dokumen dengan review yang sudah bersih
    prepo_collection.update_one({'_id': document['_id']}, {'$set': {'content': cleaned_review}})

    total_cleaned_count += cleaned_count

# Menampilkan jumlah tanda baca yang dihapus
print(f"Jumlah tanda baca yang dihapus: {total_cleaned_count}")


In [ ]:
def remove_number(text):
    modified_text = re.sub(r"\d+", " ", text)
    
    changes_count = count_changes(text, modified_text)

    return modified_text, changes_count

# Mengambil semua dokumen dari koleksi
documents = prepo_collection.find()

total_cleaned_count = 0

for document in documents:
    review = document['content']
    cleaned_review, cleaned_count = remove_number(review)
    
    # Memperbarui dokumen dengan review yang sudah bersih
    prepo_collection.update_one({'_id': document['_id']}, {'$set': {'content': cleaned_review}})

    total_cleaned_count += cleaned_count

# Menampilkan jumlah angka yang dihapus
print(f"Jumlah angka yang dihapus: {total_cleaned_count}")

In [ ]:
def remove_single_char(text):
    modified_text = re.sub(r"\b[a-zA-Z]\b", " ", text)
    
    changes_count = count_changes(text, modified_text)

    return modified_text, changes_count

# Mengambil semua dokumen dari koleksi
documents = prepo_collection.find()

total_cleaned_count = 0

for document in documents:
    review = document['content']
    cleaned_review, cleaned_count = remove_single_char(review)
    
    # Memperbarui dokumen dengan review yang sudah bersih
    prepo_collection.update_one({'_id': document['_id']}, {'$set': {'content': cleaned_review}})

    total_cleaned_count += cleaned_count

# Menampilkan jumlah karakter tunggal yang dihapus
print(f"Jumlah karakter tunggal yang dihapus: {total_cleaned_count}")


In [ ]:
def normalize_whitespace(text):
    modified_text = re.sub(r'\s+', ' ', text).strip()
    
    changes_count = count_changes(text, modified_text)

    return modified_text, changes_count

# Mengambil semua dokumen dari koleksi
documents = prepo_collection.find()

total_cleaned_count = 0

for document in documents:
    review = document['content']
    cleaned_review, cleaned_count = normalize_whitespace(review)
    
    # Memperbarui dokumen dengan review yang sudah dibenarkan
    prepo_collection.update_one({'_id': document['_id']}, {'$set': {'content': cleaned_review}})

    total_cleaned_count += cleaned_count

# Menampilkan jumlah whitespace yang dihapus
print(f"Jumlah whitespace yang dihapus: {total_cleaned_count}")


In [ ]:
import pandas as pd
import logging

# Konfigurasi logging
logging.basicConfig(filename='perubahan_normalisasi_bibit.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Fungsi untuk membaca kamus dari file CSV dan mengembalikan kamus singkatan
def load_abbreviations(csv_file, delimiter=';'):
    df = pd.read_csv(csv_file, delimiter=delimiter)
    return dict(zip(df['before'], df['after']))

# Fungsi untuk memperluas singkatan dalam teks
def expand_abbreviations(text, abbreviations):
    expanded_tokens = []
    for word in text.split():
        expanded_word = abbreviations.get(word.lower(), word)
        if expanded_word != word:
            logging.info(f"Mengganti '{word}' menjadi '{expanded_word}'")
        expanded_tokens.append(expanded_word)
    expanded_text = ' '.join(expanded_tokens)
    return expanded_text

# Mengambil kamus singkatan dari file CSV
abbreviations = load_abbreviations('kamus_normalisasi.csv')

# Mendapatkan data dari koleksi MongoDB
documents = prepo_collection.find()

# Inisialisasi variabel untuk menyimpan total jumlah kata yang telah diperluas
total_expanded_count = 0

# Loop melalui setiap dokumen dalam koleksi
for document in documents:
    # Ambil teks review dari dokumen
    review = document['content']
    
    # Perluas singkatan dalam teks review
    expanded_review = expand_abbreviations(review, abbreviations)
    
    # Hitung perbedaan jumlah kata sebelum dan setelah perluasan
    expanded_count = len(expanded_review.split()) - len(review.split())

    # Memperbarui dokumen dengan review yang sudah diperluas
    prepo_collection.update_one({'_id': document['_id']}, {'$set': {'content': expanded_review}})

    # Menambahkan jumlah kata yang telah diperluas ke total_expanded_count
    total_expanded_count += expanded_count

# Menampilkan jumlah kata yang telah diperluas
print(f"Jumlah kata yang telah diperluas: {total_expanded_count}")


In [ ]:
def remove_duplicate_chars(text):
    words = text.split()
    cleaned_words = []
    changes_count = 0
    for word in words:
        cleaned_word = re.sub(r'(\w)\1+', r'\1', word)
        changes_count += count_changes(word, cleaned_word)
        cleaned_words.append(cleaned_word)
    
    cleaned_text = " ".join(cleaned_words)

    return cleaned_text, changes_count

# Mendapatkan data dari koleksi MongoDB
cursor = prepo_collection.find()

total_changes_count = 0

for document in cursor:
    review = document['content']
    
    # Menghapus karakter ganda dalam review
    cleaned_review, changes_count = remove_duplicate_chars(review)
    
    # Memperbarui field 'review' dengan review yang sudah dibersihkan
    prepo_collection.update_one(
        {'_id': document['_id']},
        {'$set': {'content': cleaned_review}}
    )

    total_changes_count += changes_count

# Menampilkan jumlah karakter ganda yang dihapus
print(f"Jumlah karakter ganda yang dihapus: {total_changes_count}")


In [ ]:
# Mendapatkan data dari koleksi MongoDB
cursor = prepo_collection.find()

for document in cursor:
    review = document['content']
    
    # Melakukan tokenisasi menggunakan word_tokenize dari nltk
    tokens = word_tokenize(review)
    
    # Memperbarui field 'tokenized_review' dengan daftar token
    prepo_collection.update_one(
        {'_id': document['_id']},
        {'$set': {'tokenized_review': tokens}}
    )

    print(tokens)

In [ ]:
import nltk
from nltk.corpus import stopwords
# Mendapatkan data dari koleksi MongoDB
cursor = prepo_collection.find()

# Mengambil daftar stopwords dalam bahasa Indonesia
list_stopwords = stopwords.words('indonesian')

list_stopwords.extend([
    "ny", 'klo', 'aah','ah','luu','we','wee','la','tah','kalo', 'biar', 'buat', 'bilang','bagaimana','cuma', 'karena', 'nya', 'nih', 'sih','at','gitu','tapi',
    'si', 'tau', 'tuh', 'untuk', 'ya', 'iya','mah','nder','jadi', 'sudah', 'saja','wah','ini','gw', 'hehe', 'pen','nan', 'loh', 'iii','wah','si','vi',
    'eee', 'setelah', 'tynyjunaz', 'ktol', 'yang', 'padahal', 'bagetikut', 'mu', 'loh', 'sih', 'tiba', 'tari', 'saja', 'ah',
    'kah', 'sekali', 'aju', 'hey', 'gini', 'halo', 'buk', 'si', 'hai', 'halo', 'min', 'lh', 'so', 'wae', 'jos', 'kalau', 'di', 'dh', 'dah',
    'ke', 'dom', 'juga', 'tiba', 'dari', 'ni', 'kak', 'mas', 'mba', 'eh', 'sat', 'ini'
])


In [ ]:
list_stopwords = set(list_stopwords)

for document in cursor:
    review = document['tokenized_review']
    
    # Menghapus stopwords dari review menggunakan stopwords dalam bahasa Indonesia
    filtered_review = [word for word in review if word.lower() not in list_stopwords]
    
    # Memperbarui field 'filtered_review' dengan review yang sudah tidak mengandung stopwords
    prepo_collection.update_one(
        {'_id': document['_id']},
        {'$set': {'filtered_review': filtered_review}}
    )

    print (filtered_review)

In [ ]:
# Inisialisasi objek stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Mendapatkan data dari koleksi MongoDB
cursor = prepo_collection.find()

for document in cursor:
    filtered_review = document['filtered_review']
    
    # Melakukan stemming pada setiap kata dalam filtered_review
    stemmed_review = [stemmer.stem(word) for word in filtered_review]
    
    # Memperbarui field 'stemmed_review' dengan hasil stemming
    prepo_collection.update_one(
        {'_id': document['_id']},
        {'$set': {'stemmed_review': stemmed_review}}
    )

    print (stemmed_review)

In [ ]:
# Mendapatkan data dari koleksi MongoDB
cursor = prepo_collection.find()

for document in cursor:
    stemmed_review = document['stemmed_review']
    
    # Mengubah array kata hasil stemming menjadi string
    data_clean = ' '.join(stemmed_review)
    
    # Memperbarui field 'data_clean' dengan hasil yang sudah diubah
    prepo_collection.update_one(
        {'_id': document['_id']},
        {'$set': {'data_clean': data_clean}}
    )

    print (data_clean)

In [ ]:
# Mengecek dan menghapus dokumen dengan field "data_clean" yang kosong
query = {'data_clean': {'$exists': True, '$eq': ''}}
result = prepo_collection.delete_many(query)
print(f"Jumlah dokumen yang dihapus: {result.deleted_count}")

In [ ]:
# Pipeline untuk menghilangkan data duplikat berdasarkan field "data_clean"
pipeline = [
    {
        '$group': {
            '_id': '$data_clean',
            'uniqueIds': {'$addToSet': '$_id'},
            'count': {'$sum': 1}
        }
    },
    {
        '$match': {
            'count': {'$gt': 1}
        }
    }
]

# Eksekusi pipeline
duplicates = list(prepo_collection.aggregate(pipeline))

# Menghapus data duplikat
total_duplicates_removed = 0
for duplicate in duplicates:
    review_ids_to_remove = duplicate['uniqueIds'][1:]
    result = prepo_collection.delete_many({'_id': {'$in': review_ids_to_remove}})
    duplicates_removed_count = result.deleted_count
    total_duplicates_removed += duplicates_removed_count

print('Jumlah data duplikat yang dihapus:', total_duplicates_removed)


In [ ]:
cursor = prepo_collection.find()
df_bibit= pd.DataFrame(list(cursor))
df_bibit

In [ ]:
# Membuat koneksi ke database MongoDB
client = MongoClient('localhost', 27017)
db = client['google_play_store_data'] 
prepo_collection = db['02. prepo_data_bibit']  
clean_collection = db['03. clean_data_bibit'] 

# Mengambil beberapa dokumen dari koleksi lama
data = prepo_collection.find({}) 

# Menyimpan dokumen-dokumen tersebut ke dalam list
data_list = list(data)

# Menyimpan data ke dalam koleksi baru
clean_collection.insert_many(data_list)


In [ ]:
# Menghapus field yang tidak diperlukan
clean_collection.update_many({}, {'$unset': {'tokenized_review': '', 'filtered_review': '', 'stemmed_review': '', 'content': ''}})

# Menampilkan data setelah penghapusan field
cursor = clean_collection.find()
for document in cursor:
    print(document)

In [ ]:
clean_collection.update_many(
    {},
    {
        '$set': {
            'label': None
        }
    }
)

In [ ]:
client.close()